In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import gtsam
from gtsam.symbol_shorthand import X, L
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.slam.gtsam_factor_graph import GtsamFactorGraph
from lac.slam.slam import SLAM
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.slam.feature_tracker import FeatureTracker
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.utils.visualization import image_grid
from lac.util import load_data, load_stereo_images, load_images

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
left_imgs, right_imgs = load_stereo_images(data_path, start_frame=0, end_frame=10000)
images = {"FrontLeft": left_imgs, "FrontRight": right_imgs}

In [ ]:
# Initialize modules
START_FRAME = 80

tracker = FeatureTracker(cam_config)
tracker.initialize(poses[START_FRAME], left_imgs[START_FRAME], right_imgs[START_FRAME])

svo = StereoVisualOdometry(cam_config)
svo.initialize(poses[START_FRAME], left_imgs[START_FRAME], right_imgs[START_FRAME])

In [ ]:
from lac.params import FL_X, FL_Y, IMG_HEIGHT, IMG_WIDTH, STEREO_BASELINE

skew = 0.0
stereo_cal = gtsam.Cal3_S2Stereo(FL_X, FL_Y, skew, IMG_WIDTH / 2, IMG_HEIGHT / 2, STEREO_BASELINE)
stereo_noise = gtsam.noiseModel.Isotropic.Sigma(3, 2.0)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(poses[START_FRAME])))
initial_estimate.insert(X(0), gtsam.Pose3(poses[START_FRAME]))

In [ ]:
END_FRAME = 200
KEYFRAME_RATE = 10
GRAPH_UPDATE_RATE = 1

landmark_ids = set()
curr_pose = poses[START_FRAME]
svo_poses = [poses[START_FRAME]]

pose_key = 1

for frame in tqdm(range(START_FRAME + 2, END_FRAME, 2)):
    # VO
    svo.track(images["FrontLeft"][frame], images["FrontRight"][frame])
    curr_pose = svo.get_pose()
    svo_poses.append(curr_pose)

    # Feature tracking
    tracker.track_keyframe(curr_pose, images["FrontLeft"][frame], images["FrontRight"][frame])

    # Update the graph
    initial_estimate.insert(X(pose_key), gtsam.Pose3(curr_pose))

    # Add stereo factors
    disparities = tracker.prev_pts - tracker.prev_pts_right
    for i, id in enumerate(tracker.track_ids):
        if disparities[i][1] != 0:  # For now ignore matches with nonzero y disparity
            continue

        stereo_meas = gtsam.StereoPoint2(
            tracker.prev_pts[i][0],  # uL
            tracker.prev_pts_right[i][0],  # uR
            tracker.prev_pts[i][1],  # v
        )
        if id not in landmark_ids:
            landmark_ids.add(id)
            initial_estimate.insert(L(id), tracker.world_points[i])
        graph.add(
            gtsam.GenericStereoFactor3D(
                stereo_meas,
                stereo_noise,
                X(pose_key),
                L(id),
                stereo_cal,
            )
        )

    pose_key += 1

In [ ]:
print(f"Initial error: {graph.error(initial_estimate)}")
params = gtsam.LevenbergMarquardtParams()
# params.setVerbosity("TERMINATION")
params.setVerbosity("ERROR")
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()